<a href="https://colab.research.google.com/github/MuneeraAlhajri/CMT316_CW2_Muneerah/blob/main/model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
from tensorflow import keras
from keras import applications
from keras import optimizers
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils.np_utils import to_categorical
from keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from google.colab import drive

drive.mount('/content/drive')

train_path = "/content/drive/MyDrive/Data/ML Coursework 2/train/"
test_path = "/content/drive/MyDrive/Data/ML Coursework 2/test/"
img_width, img_height = 224, 224
channels = 3
batch_size = 64
num_images = 100
image_arr_size = img_width * img_height * channels


Mounted at /content/drive


In [2]:

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    validation_split=0.2,

)

valid_datagen = ImageDataGenerator(
    rescale= 1./255,
    validation_split=0.2,
)


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='training',
    shuffle= True,
    seed= 1337
)

valid_generator = valid_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='validation',
    shuffle= True,
    seed= 1337
)

Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.


In [3]:
num_classes = len(train_generator.class_indices)
train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = valid_generator.classes
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_generator.filenames)
nb_valid_samples = len(valid_generator.filenames)

In [4]:
test_datagen = ImageDataGenerator(rescale=1/255.0)
test_generator = test_datagen.flow_from_directory(
                                    test_path,
                                    target_size=(img_width, img_height),
                                    color_mode= 'rgb',
                                    batch_size=batch_size,
                                    class_mode= 'categorical',
                                    shuffle=False,)
test_generator.reset()

Found 8580 images belonging to 120 classes.


# Model

**The following code are building model, you should optimized them.**



In [5]:
InceptionV3 = InceptionV3(include_top= False, input_shape= (img_width, img_height, channels), weights= 'imagenet')
# InceptionV3.summary()

model = Sequential()

for layer in InceptionV3.layers:
    layer.trainable = False

model.add(InceptionV3)
model.add(layers.Dropout(0.3))

model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(1000,kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dense(120,activation='softmax'))
model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(train_generator, steps_per_epoch=50, epochs=10, verbose= 1, validation_data= valid_generator)

87924736/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 dropout (Dropout)           (None, 5, 5, 2048)        0         
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 batch_normalization_94 (Bat  (None, 51200)            204800    
 chNormalization)                                                
                                                                 
 dense (Dense)               (None, 1000)              51201000  
                                                                 
 batch_normalization_95 (Bat  (None, 1000)             40

In [6]:
pred = model.predict(test_generator, verbose=2)
pred = np.argmax(pred,axis=1)

135/135 - 1699s - 1699s/epoch - 13s/step


In [7]:
test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)
test_accuracy = 100*np.sum(pred==np.argmax(test_labels, axis=1))/len(pred)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 78.2984%


In [8]:
print(pred)
print(len(pred))
test_targets = np.argmax(test_labels, axis=1)
a = 0
for i in range(8580):
  if(pred[i]==test_targets[i]):
    a+=1
print(a)

[ 15   0   0 ...   2 119 119]
8580
6718


In [11]:
print(test_targets)

[  0   0   0 ... 119 119 119]


In [10]:
score = model.evaluate(test_generator, verbose=2)

135/135 - 938s - loss: 0.7713 - accuracy: 0.7830 - 938s/epoch - 7s/step


In [12]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(test_targets, pred)
print(cm)

[[ 38   0   0 ...   0   0   0]
 [  0 127   0 ...   0   0   0]
 [  0   0  39 ...   0  43   2]
 ...
 [  0   0   0 ...  68   0   0]
 [  0   0   7 ...   0  38   1]
 [  0   0  18 ...   0  28  31]]


In [14]:
from sklearn.metrics import classification_report
print(classification_report(test_targets, pred))

              precision    recall  f1-score   support

           0       0.54      0.73      0.62        52
           1       0.94      0.91      0.93       139
           2       0.54      0.42      0.48        92
           3       0.79      0.82      0.80        50
           4       0.84      0.84      0.84       109
           5       0.68      0.94      0.79       100
           6       0.93      0.93      0.93       110
           7       0.75      0.88      0.81        95
           8       0.90      0.81      0.86       113
           9       0.91      0.92      0.92       118
          10       0.93      0.91      0.92       119
          11       0.95      0.97      0.96        96
          12       0.85      0.81      0.83        75
          13       0.98      0.95      0.96        58
          14       0.88      0.81      0.84        53
          15       0.76      0.79      0.78        81
          16       0.71      0.73      0.72        55
          17       0.56    